In [4]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast

In [126]:
df5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\apoe33_notes_1.csv")

In [127]:
df5_matches = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_matches.csv")

In [128]:
unique_ids = df5_matches["PatientID"].unique()
unique_ids = list(unique_ids)
len(unique_ids)

2890

In [15]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\Sentence-Analysis\EDA\Getting_Data\keywords.csv")
regex = regex["REGEX"].to_list()

In [99]:
print(df5_matches.columns)

Index(['Unnamed: 0', 'EMPI', 'MRN', 'PatientID', 'PatientEncounterID',
       'ContactDTS', 'EncounterTypeDSC', 'NoteID', 'InpatientNoteTypeDSC',
       'buckets', 'NoteCSNID', 'LineNBR', 'NoteTXT', 'regex_location',
       'regex_sent', 'regex_match', 'match_count'],
      dtype='object')


In [132]:
cols =  ["patient_id", "encounters_with_keywords",'dementia', 'cognition',"\'cognition", 'memory', "memory\'", 'cdr', 'mmse', 'moca', 'alzheimer', 'cognitive impairment','cognitiveimpairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's", 'cortical', 'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia']
summary_stats  = pd.DataFrame(pd.np.empty((len(unique_ids), len(cols))) * pd.np.nan) 
summary_stats.columns = cols
summary_stats["patient_id"] = unique_ids
summary_stats

,patient_id,encounters_with_keywords,dementia,cognition,'cognition,memory,memory',cdr,mmse,moca,...,mci,cerebellar,neurocognitive,lewy,pick's,cortical,corticobasal,cerebral,cerebrovascular,amnesia
0,Z6352398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Z6352614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Z6352700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Z6352922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Z6352946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,Z8712370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2886,Z8713474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2887,Z8713489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2888,Z8713893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
for i in tqdm(range(len(unique_ids))):
    keyword_list = df5_matches[df5_matches['PatientID'] == unique_ids[i]]['regex_match'].values
    
    match_tuples = []
    for j in range(len(keyword_list)):
        sub = "\', \'"
        x = keyword_list[j][2:len(keyword_list[j])-2]
        if sub in x:
            x = x.split("\', \'")
        else:
            x = x.split("\", \"")
        match_tuples.append((x[0], 0))

    #print(match_tuples)

    freq = {"dementia":0
        ,"cognition":0
        ,"\'cognition":0
        ,"memory":0
        ,"memory\'":0
        ,"cdr":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        , "cognitiveimpairment":0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"cortical":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
    }

    for k, v in match_tuples:   
        if (k != ''):
            freq.update({str(k.lower()):int(freq[k.lower()] + 1)})

    #print(freq, i, "\n")

    summary_stats["encounters_with_keywords"][i] = len(df5_matches[df5_matches["PatientID"] == unique_ids[i]])

    for k, v in freq.items():
        summary_stats[str(k.lower())][i] = int(v)
        #print(k, v)

100%|██████████████████████████████████████████████████████████████████████████████| 2890/2890 [02:35<00:00, 18.56it/s]


In [145]:
keyword_list = df5_matches[df5_matches['PatientID'] == unique_ids[436]]['regex_match'].values

match_tuples = []
for j in range(len(keyword_list)):
    sub = "\', \'"
    x = keyword_list[j][2:len(keyword_list[j])-2]
    print("before: ",x)
    if sub in x:
        x = x.split("\', \'")
    else:
        x = x.split("\", \"")
    print("after: ", x)
    #
    #print(x)
    match_tuples.append((x[0], 0))

#print(match_tuples)

freq = {"dementia":0
    ,"cognition":0
    ,"\'cognition":0
    ,"memory":0
    ,"memory\'":0
    ,"cdr":0
    ,"mmse":0
    ,"moca":0
    ,"alzheimer":0
    ,'cognitive impairment':0
    , "cognitiveimpairment":0
    ,"mci":0
    ,"cerebellar":0
    ,"neurocognitive":0
    ,"lewy":0
    ,"pick's":0
    ,"cortical":0
    ,"corticobasal":0
    ,"cerebral":0
    ,"cerebrovascular":0
    ,"amnesia":0
}

for k, v in match_tuples:    
    if (k != ''):
        freq.update({str(k.lower()):int(freq[k.lower()] + 1)})

#print(freq, i, "\n")

summary_stats["encounters_with_keywords"][436] = len(df5_matches[df5_matches["PatientID"] == unique_ids[436]])

for k, v in freq.items():
    summary_stats[str(k.lower())][i] = int(v)
    #print(k, v)

before:  Memory', 'Cerebellar
after:  ['Memory', 'Cerebellar']
before:  Memory', 'Cerebellar
after:  ['Memory', 'Cerebellar']
before:  cerebral
after:  ['cerebral']
before:  mci
after:  ['mci']
before:  cerebral
after:  ['cerebral']
before:  cerebral
after:  ['cerebral']
before:  memory
after:  ['memory']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral
after:  ['cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cer

before:  memory
after:  ['memory']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral
after:  ['cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral
after:  ['cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  memory
after:  ['memory']
before:  MOCA
after:  ['MOCA']
before:  memory
after:  ['memory']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral', 'cerebral
after:  ['cerebral', 'cerebral']
before:  cerebral
after:  ['cerebral']
before:  cortical
after:  ['cortical']
before:  mCi
after:  ['mCi']
before:  dementia
after:  ['dementia']
before:  c

In [147]:
summary_stats

,patient_id,encounters_with_keywords,dementia,cognition,'cognition,memory,memory',cdr,mmse,moca,...,mci,cerebellar,neurocognitive,lewy,pick's,cortical,corticobasal,cerebral,cerebrovascular,amnesia
0,Z6352398,51.0,1.0,13.0,0.0,13.0,0.0,0.0,0.0,5.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0
1,Z6352614,18.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,0.0,...,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Z6352700,24.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0,0.0
3,Z6352922,14.0,0.0,3.0,0.0,9.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,Z6352946,16.0,0.0,4.0,0.0,8.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,Z8712370,18.0,11.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2886,Z8713474,31.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
2887,Z8713489,22.0,0.0,16.0,0.0,1.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2888,Z8713893,95.0,1.0,23.0,0.0,47.0,0.0,0.0,1.0,0.0,...,1.0,9.0,1.0,0.0,0.0,2.0,0.0,8.0,0.0,0.0


In [148]:
summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_summary_stats_updated.csv")

In [149]:
s1 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\1\apoe_1_summary_stats_updated.csv")
s2 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\2\apoe_2_summary_stats_updated.csv")
s3 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\3\apoe_3_summary_stats_updated.csv")
s4 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\4\apoe_4_summary_stats_updated.csv")
s5 = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\5\apoe_5_summary_stats_updated.csv")

In [150]:
frames = [s1,s2,s3,s4,s5]
total_summary_stats = pd.concat(frames)

In [151]:
total_summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match_csv\total_summary_stats_updated.csv")